# **Library Imports**

In [2]:
import pandas as pd
import io
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from numpy import sqrt

from matplotlib.colors import ListedColormap

from sklearn import utils
from sklearn.datasets import make_blobs
from sklearn.svm import SVR, LinearSVR, SVC
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler,LabelEncoder,StandardScaler,PowerTransformer
from sklearn.model_selection import cross_val_score,train_test_split,cross_val_predict, KFold
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import BayesianRidge

import pickle

ModuleNotFoundError: No module named 'seaborn'

# **Dataset Information.**

In [ ]:
df = pd.read_csv('/content/BodyFat_-_Extended.csv')
df = df.rename(columns=lambda x: x.strip())
print(df.head())
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print(df.info())

   BodyFat Sex  Age  Weight  Height  Neck  Chest  Abdomen    Hip  Thigh  Knee  \
0     12.3   M   23   69.97    1.72  36.2   93.1     85.2   94.5   59.0  37.3   
1      6.1   M   22   78.59    1.84  38.5   93.6     83.0   98.7   58.7  37.3   
2     25.3   M   22   69.85    1.68  34.0   95.8     87.9   99.2   59.6  38.9   
3     10.4   M   26   83.80    1.84  37.4  101.8     86.4  101.2   60.1  37.3   
4     28.7   M   24   83.58    1.81  34.4   97.3    100.0  101.9   63.2  42.2   

   Ankle  Biceps  Forearm  Wrist  
0   21.9    32.0     27.4   17.1  
1   23.4    30.5     28.9   18.2  
2   24.0    28.8     25.2   16.6  
3   22.8    32.4     29.4   18.2  
4   24.0    32.2     27.7   17.7  
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 434 entries, 0 to 433
Data columns (total 15 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   BodyFat  434 non-null    float64
 1   S

# **Bayesian Ridge Regression Algorithm Implementation.**

### **Data normalization/mapping, splitting the dataset into training and testing sets, and training the model.**


In [ ]:
df['Sex'] = df['Sex'].map({'M': 1, 'F': 0})

# Setting the features.
X = df.drop('BodyFat', axis=1)
y = df['BodyFat']

# Splitting the data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.332, random_state=42)

# Define the hyperparameter grid
param_grid = {
    'alpha_1': [1e-7, 1e-6, 1e-5, 1e-4],
    'alpha_2': [1e-7, 1e-6, 1e-5, 1e-4],
    'lambda_1': [1e-7, 1e-6, 1e-5, 1e-4],
    'lambda_2': [1e-7, 1e-6, 1e-5, 1e-4],
    'fit_intercept': [True, False],
    'compute_score': [True, False]
}

# Creating and training the Bayesian Ridge Regression model.
model = BayesianRidge()
model.fit(X_train, y_train)

BayesianRidge()

### **Evaluation of the results obtained.**


In [ ]:
# Set up GridSearchCV
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
best_model = grid_search.best_estimator_

# Using the trained model to predict values on the testing set.
y_pred_best = best_model.predict(X_test)

# Evaluating the model's mean squared error.
mse_best = mean_squared_error(y_test, y_pred_best)

# Evaluating the model's r-squared error.
r_squared_best = r2_score(y_test, y_pred_best)

print(f'Best Model Mean Squared Error: {mse_best}')
print(f'Best Model R-Squared Error: {r_squared_best}')

# Saving the model to a .sav file to be loaded in the 'Models.ipynb' file.
filename = 'bayesian_ridge_regression.sav'
pickle.dump(best_model, open(filename, 'wb'))

Best Hyperparameters: {'alpha_1': 0.0001, 'alpha_2': 1e-07, 'compute_score': True, 'fit_intercept': True, 'lambda_1': 1e-07, 'lambda_2': 0.0001}
Best Model Mean Squared Error: 15.087536010526692
Best Model R-Squared Error: 0.6411318441010325


### **Testing the model on example data.**

In [ ]:
def convert_sex(sex):
    return 1 if sex.lower() == 'm' else 0

# Function to get user input and make predictions.
def predict_body_fat(model, input_data):

    input_data['Sex'] = convert_sex(input_data['Sex'])

    # Converting input data to a DataFrame.
    input_df = pd.DataFrame([input_data])

    # Predicting the Body Fat value based on the user input.
    prediction = model.predict(input_df)

    return prediction[0]

# Asking user to input their measurment details.
sex = input("Sex (M/F): ")
age = float(input("Age: "))
weight = float(input("Weight: "))
height = float(input("Height: "))
neck = float(input("Neck circumference: "))
chest = float(input("Chest circumference: "))
abdomen = float(input("Abdomen circumference: "))
hip = float(input("Hip circumference: "))
thigh = float(input("Thigh circumference: "))
knee = float(input("Knee circumference: "))
ankle = float(input("Ankle circumference: "))
bicep = float(input("Bicep circumference: "))
forearm = float(input("Forearm circumference: "))
wrist = float(input("Wrist circumference: "))


# Creating the structure for the user input.
user_input = {
    'Sex': sex,  # Assuming 0 for male, 1 for female
    'Age': age,
    'Weight': weight,
    'Height': height,
    'Neck': neck,
    'Chest': chest,
    'Abdomen': abdomen,
    'Hip': hip,
    'Thigh': thigh,
    'Knee': knee,
    'Ankle': ankle,
    'Biceps': bicep,
    'Forearm': forearm,
    'Wrist': wrist
}

# Calling the function on the user input.
user_prediction = predict_body_fat(grid_search, user_input)

print(f'\n\nBased on the information provided, your predicted Body Fat Percentage evaluates to: {user_prediction:.2f}%')

# BodyFat Sex  Age  Weight  Height  Neck  Chest  Abdomen  Hip   Thigh  Knee  Ankle  Biceps  Forearm  Wrist
#   12.3   M   23   69.97    1.72  36.2   93.1     85.2   94.5   59.0  37.3   21.9    32.0     27.4   17.12

KeyboardInterrupt: Interrupted by user